# Dataset & Preprocessing

At the outset of our project, we needed a single, unified table that combined self-reported happiness scores with country-level development indicators. We selected:

- **World Happiness Report (2019)** from Kaggle, for its Life Ladder (happiness) scores and related psychosocial factors.  
- **World Bank Development Indicators** (WDI), for six concrete metrics (GDP per capita, GNI per capita, Gini index, life expectancy, unemployment rate, and education spending).

Below we describe how we cleaned and merged these sources, and document every variable in our final dataset.

---

## Cleaning

1. **Standardize column names**  
   - Stripped whitespace, converted all names to `snake_case`, and renamed for clarity.  
2. **Reshape WDI from wide to long**  
   - Melted the year-columns (`2015 [YR2015]`–`2019 [YR2019]`) into a two-column (`Year`, `Value`) format.  
3. **Filter to 2019 and our six series**  
   - Kept only the six codes we care about (`NY.GDP.PCAP.KN`, `NY.GNP.PCAP.KN`, `SI.POV.GINI`, `SP.DYN.LE00.IN`, `SL.UEM.TOTL.NE.ZS`, `SE.XPD.TOTL.GB.ZS`), all for year 2019.  
4. **Pivot back to wide**  
   - Turned those six long rows per country into six separate columns, resolving any duplicates by taking the first value.  
5. **Merge with Happiness**  
   - Inner-joined on `(Country, Year)` so that only countries present in both datasets remain.  
6. **Index & file format**  
   - Shifted the table’s index to start at 1.

After this pipeline, we arrived at **132 countries** × **12 variables**, ready for visualization.

---

## Variable Descriptions

Each row in the final table represents a country in 2019, with the following columns:

- **Country** (Nominal / Identifier)  
  The name of the country.

- **Year** (Interval)  
  The calendar year of the data (2019).

- **Happiness** (Continuous / Ratio)  
  The Life Ladder score from the World Happiness Report.

- **GDP_per_capita** (Continuous / Ratio)  
  Gross Domestic Product per capita (constant local currency units) from the World Bank.

- **GNI_per_capita** (Continuous / Ratio)  
  Gross National Income per capita (constant local currency units) from the World Bank.

- **Edu_expenditure_pct** (Continuous / Ratio)  
  Government expenditure on education, as a percentage of GDP, from the World Bank.

- **Gini_index** (Continuous / Ratio)  
  Gini coefficient measuring income inequality (0 = perfect equality, 100 = maximum inequality).

- **Unemployment_rate** (Continuous / Ratio)  
  Unemployment, total (% of total labour force) from the World Bank.

- **Life_expectancy** (Continuous / Ratio)  
  Life expectancy at birth, in years, from the World Bank.  


---


In [1]:
import pandas as pd
import re

#
df_happy = pd.read_csv('2019happy.csv')
df_happy.columns = df_happy.columns.str.strip()
df_happy = df_happy.rename(columns={
    'Country or region': 'Country',
    'Score':             'Happiness',
    'Social support':    'Social_support'
})
df_happy['Year'] = 2019

#
df = pd.read_csv('WDI_new.csv')
df.columns = df.columns.str.strip()
df = df.rename(columns={'Country Name':'Country','Series Code':'Series_Code'})

#
year_cols = [c for c in df.columns if re.match(r'\d{4} \[YR\d{4}\]', c)]
year_map = {c: c.split(' ')[0] for c in year_cols}
df = df.rename(columns=year_map)
df_long = df.melt(
    id_vars=['Country','Series_Code'],
    value_vars=list(year_map.values()),
    var_name='Year', value_name='Value'
)
df_long['Year'] = df_long['Year'].astype(int)
df_long['Value'] = pd.to_numeric(df_long['Value'], errors='coerce')

#
wanted = {
    "NY.GDP.PCAP.KN":    "GDP_per_capita",
    "NY.GNP.PCAP.KN":    "GNI_per_capita",
    "SI.POV.GINI":       "Gini_index",
    "SP.DYN.LE00.IN":    "Life_expectancy",
    "SL.UEM.TOTL.NE.ZS": "Unemployment_rate",
    "SE.XPD.TOTL.GB.ZS": "Edu_expenditure_pct" 
}
w19 = df_long[(df_long['Year']==2019) & 
             (df_long['Series_Code'].isin(wanted))]
wide = (
    w19.pivot_table(
        index=['Country','Year'],
        columns='Series_Code',
        values='Value',
        aggfunc='first'
    )
    .rename(columns=wanted)
    .reset_index()
)

# 
df_final = pd.merge(df_happy[['Country','Year','Happiness']],
                    wide, on=['Country','Year'], how='inner')

df_final.index = df_final.index + 1
display(df_final.head())

,Country,Year,Happiness,Gini_index,Unemployment_rate,Life_expectancy
1,Finland,2019,7.769,27.7,6.695,81.982927
2,Denmark,2019,7.600,27.7,5.018,81.451220
3,Norway,2019,7.554,27.7,3.684,82.958537
4,Iceland,2019,7.494,NaN,3.507,83.163415
5,Netherlands,2019,7.488,29.2,3.379,82.112195
